# Learning the topology of a Bayesian Network from a database of cases using the K2 algorithm

In [3]:
library(tidyverse)

# Installed just by typing install.packages('bnlearn')
library(bnlearn)

# This was a bit more complicated to install:
# install.packages("BiocManager")
# BiocManager::install("Rgraphviz")
library(Rgraphviz)

In [22]:
# Robinson recursive formula to compute the number of possible belief-network stuctures
# that contains n nodes
# From: A Bayesian Method for the Induction of Probabilistic Networks from Data, p. 319
n.networks.structure <- function(n)
    {
        if(n <= 1){return(1)}
        else
            {
                i <- 1
                res <- 0
                while(i <= n)
                    {
                        res <- res + ((i%%2)*2 - 1)*(choose(n,i)*(2^(i*(n-i)))*n.networks.structure(n-i))
                        i <- i + 1
                    }

                return(res)
            }
    }

for(i in 1:8)
    {
        cat('For n=',i,'the number of possible structures is: ',n.networks.structure(i),'\n')
    }


For n= 1 the number of possible structures is:  1 
For n= 2 the number of possible structures is:  3 
For n= 3 the number of possible structures is:  25 
For n= 4 the number of possible structures is:  543 
For n= 5 the number of possible structures is:  29281 
For n= 6 the number of possible structures is:  3781503 
For n= 7 the number of possible structures is:  1138779265 
For n= 8 the number of possible structures is:  783702329343 


# Dataset

#### Info:
https://rdrr.io/cran/bnlearn/man/alarm.html \
https://www.bnlearn.com/bnrepository/discrete-medium.html#alarm